In [5]:
import mysql.connector as connector

mydb = connector.connect(
    host = "54.218.75.107",
    user = "mytestuser",
    passwd = "a8336353",
    database = "rmp_uci")

mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM professors")
myresult = mycursor.fetchall()

first_id = {}
last_id = {}
for x in myresult:
    if x[1] in first_id:
        first_id[x[1]].append(x[0])
    else:
        first_id[x[1]] = [x[0]]
    if x[3] in last_id:
        last_id[x[3]].append(x[0])
    else:
        last_id[x[3]] = [x[0]]

mydb.close()


In [6]:
import pandas as pd

csv_data = pd.read_csv('res2A.csv', encoding = "ANSI")

result_id = []
for name in csv_data["教授姓名"]:
    temp = str(name).replace(",", " ")
    temp_list = temp.split()
    if len(temp_list) == 0:
        result_id.append(-1)
        continue
    pid = first_id.get(temp_list[0], [])
    
    if len(pid) == 1:
        result_id.append(pid[0])
        continue
    
    if len(pid) == 0:
        pid = last_id.get(temp_list[0], [])
        if len(pid) == 0:
            result_id.append(-1)
            continue
        if len(pid) == 1:
            result_id.append(pid[0])
            continue
        result_id.append(-1)
        continue
    
    # len(pid) >= 2
    if len(temp_list) > 1:
        pid2 = last_id.get(temp_list[-1], [])
        if len(pid2) == 0:
            result_id.append(-1)
            continue
        if len(pid2) == 1:
            result_id.append(pid2[0])
            continue
        flag = True
        for p in pid:
            if p in pid2:
                result_id.append(p)
                flag = False
        if flag: result_id.append(-1)
    else: result_id.append(-1)

seri = pd.Series(result_id)
csv_data['pid'] = seri.values
csv_data.to_csv("out2.csv", encoding = "ANSI")

In [7]:
import mysql.connector as connector

mydb = connector.connect(
    host = "54.218.75.107",
    user = "mytestuser",
    passwd = "a8336353",
    database = "rmp_uci")

mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM courses")
myresult = mycursor.fetchall()

course_id = {}
for x in myresult:
    course_id[x[1]] = x[0]

mydb.close()

In [8]:
import pandas as pd

csv_data = pd.read_csv('out2.csv', encoding = "ANSI")
result_id = []
for cname in csv_data['课程编号及名称 (e.g. ICS 6B Boolean Algebra & Logic )']:
    index = 0
    cname = str(cname).upper()
    for i in range(len(cname)):
        if not cname[i].isdigit():
            i = i + 1
        else:
            index = i
            break
    dept = cname[:index].rstrip()
    if len(cname) > 7 and cname[:7] == 'IN4MATX':
        index = 7
        dept = 'INF'
    elif len(cname) > 11 and cname[:11] == 'INFORMATICS':
        index = 11
        dept = 'INF'
    elif len(cname) > 7 and cname[:7] == 'COMPSCI':
        index = 7
        dept = 'CS'
    num = cname[index:].lstrip()
    for i in range(len(num)):
        if num[i] == ' ':
            num = num[:i]
            break
    l = 3 - len(num)
    if l > 0:
        num = '0' * l + num
    temp = dept + ' ' + num
    result_id.append(course_id.get(temp, -1))
    
seri = pd.Series(result_id)
csv_data['cid'] = seri.values
csv_data.to_csv("out3.csv", encoding = "ANSI")